## VGG
vggNet 是第一个真正意义上的深层网络结构，其是 ImageNet2014年的冠军
vgg 几乎全部使用 3 x 3 的卷积核以及 2 x 2 的池化层，使用小的卷积核进行多层的堆叠和一个大的卷积核的感受野是相同的，同时小的卷积核还能减少参数，同时可以有更深的结构。

vgg 的一个关键就是使用很多层 3 x 3 的卷积然后再使用一个最大池化层，这个模块被使用了很多次，下面我们照着这个结构来写一写

In [1]:
import tensorflow as tf
from utils import cifar10_input

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 我们定义一个批次有64个样本
batch_size = 64

In [3]:
# # 先下载数据
#链接：https://pan.baidu.com/s/1fbThmv8LDhLRemA62m3nlQ 
#提取码：j1d8 
# 解压文件，建立目录'./cifar10_data/cifar-10-batches-bin/'
train_imgs, train_labels, val_imgs, val_labels = cifar10_input.load_data()

In [5]:
from utils.layers import conv, max_pool, fc
from utils.learning import train

- `vgg`是一个不断堆叠的网络结构, 我们从它重复的最小单元写起

In [6]:
def vgg_block(inputs, num_convs, out_depth, scope='vgg_block', reuse=None):
    """构建vgg_block.
    
    一个 vgg_block 由`num_convs`个卷积层和一个最大值池化层构成.
    
    Args:
        inputs: 输入
        num_convs: 这一个block里卷积层的个数
        out_depth: 每一个卷积层的卷积核个数
        scope: 变量域名
        reuse: 是否复用
    """
    in_depth = inputs.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope, reuse=reuse) as sc:
        net = inputs
        # 循环定义`num_convs`个卷积层
        for i in range(num_convs):
            net = conv(net, ksize=[3, 3], out_depth=out_depth, strides=[1, 1], padding='SAME', scope='conv%d' % i, reuse=reuse)
        net = max_pool(net, [2, 2], [2, 2], name='pool')
        
        return net

- 然后我们把很多个不同的`vgg_block`堆叠在一起

In [7]:
def vgg_stack(inputs, num_convs, out_depths, scope='vgg_stack', reuse=None):
    """构建vgg_stack.
    
    一个 vgg_stack 将若干个不同的`vgg_block`进行`stack`(堆叠)
    
    Args:
        inputs: 输入
        num_convs: 每一个block里卷积层的个数, 列表. 如`[1, 2, 3]`
        out_depths: 每一个block的卷积核个数, 列表, 如`[64, 128, 256]`
        scope: 变量域名
        reuse: 是否复用
    """
    with tf.variable_scope(scope, reuse=reuse) as sc:
        net = inputs
        for i, (n, d) in enumerate(zip(num_convs, out_depths)):
            net = vgg_block(net, n, d, scope='block%d' % i)
        return net

- 最后我们在通过几个全连接层将`vgg`搭建完成

In [8]:
def vgg(inputs, num_convs, out_depths, num_outputs, scope='vgg', reuse=None):
    """构建vgg.
    
    一个 vgg 先经过`vgg_stack`后再连接两个全连接层.
    
    Args:
        inputs: 输入
        num_convs: 每一个 vgg_block 的卷积层的个数
        out_depths: 每一个 vgg_block 卷积核个数
        num_outputs: 最后输出向量的维数
        scope: 变量域名
        reuse: 是否复用
    """
    with tf.variable_scope(scope, reuse=reuse) as sc:
        net = vgg_stack(inputs, num_convs, out_depths)
        with tf.variable_scope('classification'):
            net = tf.reshape(net, (batch_size, -1))
            net = fc(net, 100, scope='fc1')
            net = fc(net, num_outputs, act=tf.identity, scope='classification')
        
        return net

In [9]:
train_out = vgg(train_imgs, (1, 1, 2, 2, 2), (64, 128, 256, 512, 512), 10)
# 复用上面的参数
val_out = vgg(val_imgs, (1, 1, 2, 2, 2), (64, 128, 256, 512, 512), 10, reuse=True)

#### 定义损失计算`op`

In [10]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

#### 定义正确率计算`op`

In [11]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('val'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

#### 构建训练`op`

In [12]:
lr = 0.01

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)
train_op = opt.minimize(train_loss)

#### 开始训练

In [13]:
train(train_op, train_loss, train_acc, val_loss, val_acc, 20000, batch_size)

[train]: step 0 loss = 19.6095 acc = 0.0938 (0.0088 / batch)
[val]: step 0 loss = 14.9871 acc = 0.0938
[train]: step 1000 loss = 1.3572 acc = 0.5312 (0.0261 / batch)
[train]: step 2000 loss = 1.0598 acc = 0.6719 (0.0244 / batch)
[train]: step 3000 loss = 0.7525 acc = 0.7188 (0.0249 / batch)
[train]: step 4000 loss = 0.5278 acc = 0.8594 (0.0256 / batch)
[val]: step 4000 loss = 0.8461 acc = 0.7188
[train]: step 5000 loss = 0.5448 acc = 0.8281 (0.0257 / batch)
[train]: step 6000 loss = 0.4474 acc = 0.8281 (0.0260 / batch)
[train]: step 7000 loss = 0.4204 acc = 0.8750 (0.0256 / batch)
[train]: step 8000 loss = 0.2745 acc = 0.9219 (0.0267 / batch)
[val]: step 8000 loss = 0.8315 acc = 0.7656
[train]: step 9000 loss = 0.2414 acc = 0.9062 (0.0271 / batch)
[train]: step 10000 loss = 0.0605 acc = 0.9844 (0.0264 / batch)
[train]: step 11000 loss = 0.1731 acc = 0.9688 (0.0264 / batch)
[train]: step 12000 loss = 0.1011 acc = 0.9531 (0.0252 / batch)
[val]: step 12000 loss = 0.7041 acc = 0.7969
[trai

可以看到, 20000步的训练后, `VGG`在训练集和测试集分别达到了`0.97`和`0.75`的准确率